In [1]:
import numpy as np
from qiskit import QuantumCircuit
from zne_entanglement_iqm import run_on_iqm_hardware

def hardys_circuit(theta0: float, theta1: float) -> QuantumCircuit:
    """
    q0, q1 = the two "path" qubits
    q2 = annihilation-flag ancilla
    classical bits map i <- qubit i
    """
    qc = QuantumCircuit(3, 3)

    # First beam splitters
    qc.ry(theta0, 0)
    qc.ry(theta1, 1)

    # Annihilation mark if both are in |1>
    qc.ccx(0, 1, 2)

    # Second beam splitters
    qc.ry(np.pi - theta0, 0)
    qc.ry(np.pi - theta1, 1)

    qc.measure([0, 1, 2], [0, 1, 2])
    return qc

def hardy_gamma_from_counts(counts: dict) -> float:
    """
    gamma = P(q0=0,q1=0 | q2=0)

    With measure(qi -> ci), Qiskit-style count keys are typically c2c1c0,
    i.e. bitstring[0]=c2 (q2), bitstring[1]=c1 (q1), bitstring[2]=c0 (q0).
    """
    total_no_annih = 0
    both_dark_given_no_annih = 0

    for bitstring, ct in counts.items():
        b = bitstring.replace(" ", "")
        if len(b) < 3:
            continue
        q2, q1, q0 = b[0], b[1], b[2]

        if q2 == "0":
            total_no_annih += ct
            if q0 == "0" and q1 == "0":
                both_dark_given_no_annih += ct

    return (both_dark_given_no_annih / total_no_annih) if total_no_annih else 0.0

def run_hardy_on_iqm(
    *,
    server_url: str,
    quantum_computer: str,
    token: str,
    backend_name=None,
    shots: int = 4000,
    theta: float = 0.575 * np.pi,
    optimization_level: int = 1,
    seed_transpiler: int = 1,
):
    qc = hardys_circuit(theta, theta)

    # use YOUR proven-good runner
    backend, result = run_on_iqm_hardware(
        qc,
        server_url=server_url,
        quantum_computer=quantum_computer,
        token=token,
        backend_name=backend_name,
        shots=shots,
        optimization_level=optimization_level,
        seed_transpiler=seed_transpiler,
    )

    # IQM style: index the experiment
    counts = result.get_counts(0)

    gamma = hardy_gamma_from_counts(counts)
    return backend, qc, counts, gamma


In [2]:
SERVER_URL = "https://resonance.meetiqm.com"
TOKEN = "rkSL242xyAzLHqJQrxHvHs2nREy4AbLE2aQcsA3unuoBnBZTQNV6spN3ItLtE2jJ"

backend, qc, counts, gamma = run_hardy_on_iqm(
    server_url=SERVER_URL,
    quantum_computer="emerald",
    token=TOKEN,
    shots=5000,
)

print("Backend:", backend)
print("Counts:", counts)
print("Estimated Hardy gamma:", gamma)

/home/amaciejunes/Desktop/IQM/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Backend: <iqm.qiskit_iqm.iqm_provider.IQMBackend object at 0x737337776ff0>
Counts: {'111': 779, '011': 1725, '010': 615, '110': 432, '101': 466, '001': 431, '000': 250, '100': 302}
Estimated Hardy gamma: 0.08275405494869248
